In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
from scipy.stats import spearmanr, pearsonr
from joblib import Parallel, delayed
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
rna = pd.read_csv('../use_data/rna_data.csv', index_col = 0)
#relevant_genes = np.array(pd.read_csv('../use_data/relevant_genes.csv'))
#rna = rna.iloc[:,np.isin(rna.columns, relevant_genes)]
            
rna['cell_line'] = rna.index
rna

,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,H3C3,AC098582.1,DUS4L-BCAP29,C8orf44-SGK3,ELOA3B,NPBWR1,ELOA3D,ELOA3,CDR1,cell_line
ACH-001113,4.331992,0.000000,7.364660,2.792855,4.471187,0.028569,1.226509,3.044394,6.500005,4.739848,...,0.189034,0.201634,2.130931,0.555816,0.000000,0.275007,0.0,0.000000,0.000000,ACH-001113
ACH-001289,4.567424,0.584963,7.106641,2.543496,3.504620,0.000000,0.189034,3.813525,4.221877,3.481557,...,1.049631,0.321928,1.464668,0.632268,0.000000,0.014355,0.0,0.000000,0.000000,ACH-001289
ACH-001339,3.150560,0.000000,7.379118,2.333424,4.228049,0.056584,1.310340,6.687201,3.682573,3.273516,...,1.097611,0.831877,2.946731,0.475085,0.000000,0.084064,0.0,0.000000,0.042644,ACH-001339
ACH-001538,5.085340,0.000000,7.154211,2.545968,3.084064,0.000000,5.868390,6.165309,4.489928,3.956986,...,0.632268,0.298658,1.641546,0.443607,0.000000,0.028569,0.0,0.000000,0.000000,ACH-001538
ACH-000242,6.729417,0.000000,6.537917,2.456806,3.867896,0.799087,7.208478,5.570159,7.127117,4.568032,...,2.358959,0.084064,1.910733,0.000000,0.000000,0.464668,0.0,0.000000,0.000000,ACH-000242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-000285,0.056584,0.000000,6.604368,3.266037,4.973152,0.411426,0.097611,0.704872,4.829850,5.178715,...,0.084064,1.310340,3.039138,0.344828,0.000000,0.000000,0.0,0.475085,0.042644,ACH-000285
ACH-002669,3.111031,0.000000,7.031329,1.541019,3.664483,0.014355,3.624101,6.805421,4.472488,4.397118,...,0.400538,0.356144,1.327687,0.000000,0.000000,0.014355,0.0,0.000000,0.000000,ACH-002669
ACH-001858,4.390943,0.000000,7.013239,1.887525,3.252476,0.028569,3.286881,6.902194,5.410748,3.401903,...,0.400538,0.613532,1.992768,0.704872,0.000000,1.464668,0.0,0.000000,0.526069,ACH-001858
ACH-001997,5.057450,0.000000,7.815191,2.538538,3.893362,0.028569,4.079805,6.971659,4.469886,3.463361,...,0.847997,1.292782,2.153805,0.687061,0.000000,0.000000,0.0,0.000000,0.000000,ACH-001997


In [3]:
fold = 4

In [4]:


train_test_names = pd.read_csv('../results/train_test_splits/train_test_names' +str(fold) +'.csv', index_col=0)
train_names = train_test_names[train_test_names['train_test']=='train']['cell_line']
test_names = train_test_names[train_test_names['train_test']=='test']['cell_line']

train_rna = rna[np.isin(rna.index, train_names)]
test_rna = rna[np.isin(rna.index, test_names)]

In [5]:
experiments = pd.read_csv('../use_data/prediction_targets.csv', index_col = 0)
train_experiments = experiments[np.isin(experiments['cell_line'], train_names)]
test_experiments = experiments[np.isin(experiments['cell_line'], test_names)]
train_experiments
print(test_experiments.shape,train_experiments.shape)

(124772, 8) (424780, 8)


In [6]:

drugs = experiments['DRUG'].drop_duplicates()
drugs

0          CYTARABINE
408        EPINASTINE
820       FLOXURIDINE
1289       VALRUBICIN
1759        ADAPALENE
             ...     
631598    PARDOPRUNOX
632984      RUBITECAN
634415         XL-647
635843      VERUBULIN
636320         VE-822
Name: DRUG, Length: 1135, dtype: object

In [7]:
experiments

,cell_line,DRUG,auc,logauc,auc_per_drug,means,stds,range
0,ACH-000879,CYTARABINE,1.677789,0.517477,2.367402,0.228684,0.121987,0.782691
1,ACH-000320,CYTARABINE,1.240300,0.215353,-0.109281,0.228684,0.121987,0.782691
2,ACH-001145,CYTARABINE,1.472333,0.386848,1.296560,0.228684,0.121987,0.782691
3,ACH-000873,CYTARABINE,1.207160,0.188270,-0.331294,0.228684,0.121987,0.782691
4,ACH-000855,CYTARABINE,1.229332,0.206471,-0.182097,0.228684,0.121987,0.782691
...,...,...,...,...,...,...,...,...
700916,ACH-001321,BREQUINAR,0.987409,-0.012671,0.979322,-0.184222,0.175173,1.078361
700917,ACH-001321,AZD8931,1.000000,0.000000,0.879937,-0.199376,0.226580,2.297688
700918,ACH-001321,AZD2014,1.000000,0.000000,1.643680,-0.279217,0.169873,1.025072
700919,ACH-001321,MOTESANIB,0.888088,-0.118685,-0.367226,-0.084827,0.092199,0.957449


In [8]:
def crossval(drug, train_experiments, test_experiments, train_rna, test_rna):
    train_experiments_now = train_experiments[train_experiments['DRUG']==drug]
    test_experiments_now = test_experiments[test_experiments['DRUG']==drug]

    train_data_and_labels = train_rna.merge(train_experiments_now, left_index=True, right_on = 'cell_line', how='inner')
    train_data = train_data_and_labels.iloc[:,1:-9]  
    train_label = train_data_and_labels['auc_per_drug']
    
    test_data_and_labels = test_rna.merge(test_experiments_now, left_index=True, right_on = 'cell_line', how='inner')
    cell_line = test_data_and_labels['cell_line']

    test_data = test_data_and_labels.iloc[:,1:-9]  
    print(train_data.shape, test_data.shape)
    test_label = test_data_and_labels['auc_per_drug']
    
    lasso = Lasso(alpha=0.1, max_iter=4000)    
    lasso_low = Lasso(alpha=1e-4, max_iter=4000)    
    
    ridge = Ridge(alpha=0.1)
    EN = ElasticNet(alpha=0.1)
    #linear = LinearRegression()
    svr = SVR(kernel='rbf')
    kernel_ridge = KernelRidge(alpha=0.1)

    
    lasso.fit(X = train_data, y = train_label)
    prediction_lasso = lasso.predict(test_data)
    
    ridge.fit(X = train_data, y = train_label)
    prediction_ridge = ridge.predict(test_data)
    
    kernel_ridge.fit(X = train_data, y = train_label)
    prediction_kernel_ridge = kernel_ridge.predict(test_data)
    
    EN.fit(X = train_data, y = train_label)
    prediction_EN = EN.predict(test_data)
    
    svr.fit(X = train_data, y = train_label)
    prediction_svr = svr.predict(test_data)
    
    return pd.DataFrame({'label': test_label, 
                         'lasso': prediction_lasso, 

                         'ridge': prediction_ridge, 
                         'kernel_ridge': prediction_kernel_ridge, 

                         'elastic': prediction_EN, 
                         'svr': prediction_svr, 
                         'drugs':drug, 
                         'fold': fold, 
                         'cells': cell_line})


In [9]:
crossval(np.array(drugs)[0], train_experiments=train_experiments, test_experiments=test_experiments, train_rna=train_rna, test_rna=test_rna )

(303, 19193) (95, 19193)


,label,lasso,ridge,kernel_ridge,elastic,svr,drugs,fold,cells
365,-0.116869,0.023422,-0.505573,-0.503422,-0.065156,0.054355,CYTARABINE,4,ACH-000242
379,1.439482,-0.313474,0.497121,0.495682,-0.003201,0.057386,CYTARABINE,4,ACH-000461
197,1.384772,-0.122461,-0.255654,-0.257367,-0.350187,-0.016658,CYTARABINE,4,ACH-000792
307,-0.537915,0.334100,-0.205463,-0.204605,0.148307,-0.046691,CYTARABINE,4,ACH-000769
342,-0.419810,0.142987,0.480262,0.478431,-0.325865,0.004437,CYTARABINE,4,ACH-000587
...,...,...,...,...,...,...,...,...,...
189,0.207254,-0.606462,-0.686485,-0.683746,-0.596137,-0.238326,CYTARABINE,4,ACH-000549
268,1.093199,-0.197359,-0.595856,-0.595394,-0.719423,-0.009000,CYTARABINE,4,ACH-000775
278,-0.796194,-0.261117,0.472014,0.478346,-0.014398,-0.088131,CYTARABINE,4,ACH-000097
364,-0.046294,0.081474,0.126484,0.121959,-0.288291,-0.246924,CYTARABINE,4,ACH-000972


In [10]:
#all_tests = [crossval(drug, train_experiments=train_experiments, test_experiments=test_experiments, train_rna=train_rna, test_rna=test_rna ) for drug in tqdm(np.array(drugs))]

In [11]:
all_tests = Parallel(n_jobs=10)(delayed(crossval)(drug,train_experiments, test_experiments, train_rna, test_rna ) for drug in tqdm(np.array(drugs)[:]))
all_tests

  1%|▎                                        | 10/1135 [00:00<00:24, 45.59it/s]/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy vers

/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
  4%|█▊                                       | 50/1135 [00:32<13:40,  1.32it/s]/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/

/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret 

/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret 

/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret 

...

/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
 90%|███████████████████████████████████    | 1020/1135 [14:54<01:48,  1.06it/s]/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/

/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret 

/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret 

/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
 99%|██████████████████████████████████████▍| 1120/1135 [16:32<00:14,  1.03it/s]/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/

[        label     lasso     ridge  kernel_ridge   elastic       svr   
 365 -0.116869  0.023422 -0.505573     -0.503422 -0.065156  0.054355  \
 379  1.439482 -0.313474  0.497121      0.495682 -0.003201  0.057386   
 197  1.384772 -0.122461 -0.255654     -0.257367 -0.350187 -0.016658   
 307 -0.537915  0.334100 -0.205463     -0.204605  0.148307 -0.046691   
 342 -0.419810  0.142987  0.480262      0.478431 -0.325865  0.004437   
 ..        ...       ...       ...           ...       ...       ...   
 189  0.207254 -0.606462 -0.686485     -0.683746 -0.596137 -0.238326   
 268  1.093199 -0.197359 -0.595856     -0.595394 -0.719423 -0.009000   
 278 -0.796194 -0.261117  0.472014      0.478346 -0.014398 -0.088131   
 364 -0.046294  0.081474  0.126484      0.121959 -0.288291 -0.246924   
 372 -1.083521 -0.456559 -0.841690     -0.846635 -1.149424 -0.253130   
 
           drugs  fold       cells  
 365  CYTARABINE     4  ACH-000242  
 379  CYTARABINE     4  ACH-000461  
 197  CYTARABINE     4 

In [12]:
all_tests_combined = pd.concat(all_tests, axis=0)
all_tests_combined

,label,lasso,ridge,kernel_ridge,elastic,svr,drugs,fold,cells
365,-0.116869,0.023422,-0.505573,-0.503422,-0.065156,0.054355,CYTARABINE,4,ACH-000242
379,1.439482,-0.313474,0.497121,0.495682,-0.003201,0.057386,CYTARABINE,4,ACH-000461
197,1.384772,-0.122461,-0.255654,-0.257367,-0.350187,-0.016658,CYTARABINE,4,ACH-000792
307,-0.537915,0.334100,-0.205463,-0.204605,0.148307,-0.046691,CYTARABINE,4,ACH-000769
342,-0.419810,0.142987,0.480262,0.478431,-0.325865,0.004437,CYTARABINE,4,ACH-000587
...,...,...,...,...,...,...,...,...,...
636635,-0.587709,-0.056196,0.093285,0.069228,0.267646,0.058902,VE-822,4,ACH-000775
636647,0.930052,0.537725,0.792019,0.726443,0.591108,0.462371,VE-822,4,ACH-000097
636752,-0.472742,-0.594828,-1.215951,-1.167208,-0.887542,-0.570934,VE-822,4,ACH-000972
636777,-0.137179,0.471677,0.139121,0.104883,0.172540,0.240451,VE-822,4,ACH-000791


In [13]:
all_tests_combined.to_csv('../results/other_models/lasso'+str(fold)+'.csv')

(303, 19193) (95, 19193)
(336, 19193) (104, 19193)
(300, 19193) (87, 19193)
(663, 19193) (188, 19193)
(318, 19193) (90, 19193)
(337, 19193) (99, 19193)
(327, 19193) (102, 19193)
(280, 19193) (90, 19193)
(339, 19193) (96, 19193)
(359, 19193) (103, 19193)
(350, 19193) (99, 19193)
(311, 19193) (89, 19193)
(654, 19193) (189, 19193)
(305, 19193) (96, 19193)
(321, 19193) (96, 19193)
(344, 19193) (103, 19193)
(689, 19193) (198, 19193)
(689, 19193) (193, 19193)
(313, 19193) (80, 19193)
(308, 19193) (86, 19193)
(649, 19193) (183, 19193)
(308, 19193) (93, 19193)
(285, 19193) (83, 19193)
(301, 19193) (92, 19193)
(294, 19193) (84, 19193)
(250, 19193) (72, 19193)
(275, 19193) (79, 19193)
(282, 19193) (76, 19193)
(121, 19193) (32, 19193)
(234, 19193) (71, 19193)
(259, 19193) (76, 19193)
(221, 19193) (67, 19193)
(305, 19193) (82, 19193)
(312, 19193) (87, 19193)
(548, 19193) (154, 19193)
(260, 19193) (79, 19193)
(626, 19193) (186, 19193)
(636, 19193) (189, 19193)
(70, 19193) (30, 19193)
(317, 19193) (

(656, 19193) (193, 19193)
(319, 19193) (90, 19193)
(704, 19193) (210, 19193)
(354, 19193) (105, 19193)
(352, 19193) (104, 19193)
(321, 19193) (93, 19193)
(308, 19193) (89, 19193)
(675, 19193) (196, 19193)
(343, 19193) (97, 19193)
(351, 19193) (100, 19193)
(706, 19193) (206, 19193)
(682, 19193) (199, 19193)
(351, 19193) (102, 19193)
(341, 19193) (102, 19193)
(696, 19193) (205, 19193)
(289, 19193) (74, 19193)
(315, 19193) (87, 19193)
(317, 19193) (91, 19193)
(305, 19193) (79, 19193)
(312, 19193) (94, 19193)
(306, 19193) (95, 19193)
(230, 19193) (76, 19193)
(317, 19193) (96, 19193)
(278, 19193) (80, 19193)
(294, 19193) (81, 19193)
(567, 19193) (173, 19193)
(244, 19193) (74, 19193)
(235, 19193) (71, 19193)
(253, 19193) (71, 19193)
(303, 19193) (78, 19193)
(286, 19193) (78, 19193)
(290, 19193) (86, 19193)
(214, 19193) (60, 19193)
(195, 19193) (51, 19193)
(284, 19193) (83, 19193)
(267, 19193) (76, 19193)
(537, 19193) (148, 19193)
(700, 19193) (207, 19193)
(324, 19193) (96, 19193)
(679, 19193

(317, 19193) (97, 19193)
(351, 19193) (106, 19193)
(349, 19193) (105, 19193)
(319, 19193) (93, 19193)
(330, 19193) (99, 19193)
(317, 19193) (97, 19193)
(327, 19193) (97, 19193)
(356, 19193) (106, 19193)
(348, 19193) (102, 19193)
(358, 19193) (104, 19193)
(346, 19193) (100, 19193)
(358, 19193) (104, 19193)
(323, 19193) (96, 19193)
(355, 19193) (105, 19193)
(336, 19193) (101, 19193)
(359, 19193) (106, 19193)
(355, 19193) (104, 19193)
(698, 19193) (200, 19193)
(693, 19193) (205, 19193)
(304, 19193) (84, 19193)
(310, 19193) (84, 19193)
(670, 19193) (190, 19193)
(320, 19193) (97, 19193)
(272, 19193) (77, 19193)
(225, 19193) (65, 19193)
(230, 19193) (69, 19193)
(121, 19193) (37, 19193)
(241, 19193) (72, 19193)
(72, 19193) (17, 19193)
(255, 19193) (76, 19193)
(228, 19193) (69, 19193)
(248, 19193) (74, 19193)
(251, 19193) (75, 19193)
(291, 19193) (79, 19193)
(613, 19193) (175, 19193)
(277, 19193) (80, 19193)
(178, 19193) (49, 19193)
(634, 19193) (185, 19193)
(329, 19193) (99, 19193)
(340, 1919

(306, 19193) (97, 19193)
(306, 19193) (90, 19193)
(351, 19193) (105, 19193)
(321, 19193) (96, 19193)
(317, 19193) (92, 19193)
(296, 19193) (86, 19193)
(681, 19193) (204, 19193)
(315, 19193) (98, 19193)
(333, 19193) (99, 19193)
(327, 19193) (95, 19193)
(326, 19193) (90, 19193)
(349, 19193) (101, 19193)
(321, 19193) (96, 19193)
(681, 19193) (196, 19193)
(717, 19193) (213, 19193)
(355, 19193) (104, 19193)
(347, 19193) (101, 19193)
(349, 19193) (102, 19193)
(653, 19193) (188, 19193)
(306, 19193) (88, 19193)
(310, 19193) (88, 19193)
(643, 19193) (196, 19193)
(304, 19193) (94, 19193)
(267, 19193) (77, 19193)
(274, 19193) (79, 19193)
(595, 19193) (173, 19193)
(232, 19193) (69, 19193)
(257, 19193) (76, 19193)
(518, 19193) (147, 19193)
(312, 19193) (87, 19193)
(225, 19193) (63, 19193)
(238, 19193) (67, 19193)
(195, 19193) (51, 19193)
(281, 19193) (82, 19193)
(284, 19193) (83, 19193)
(512, 19193) (145, 19193)
(345, 19193) (104, 19193)
(342, 19193) (101, 19193)
(344, 19193) (104, 19193)
(216, 191